<a href="https://colab.research.google.com/github/ZsofiaK/masterthesis/blob/main/Implementation/Pipeline/Results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Results collection

This notebook collects and presents the results of the experiments.

## Preliminaries

In [ ]:
# Notebook parameters.

n_bootstrap = 100   # Number of bootstrapped samples to take.

In [ ]:
# Specifying directories where the results are stored.
parameters_drive_dir = '/content/drive/MyDrive/UvA/M Thesis/Data/Results/Parameters'    # Best model parameters
predictions_drive_dir = '/content/drive/MyDrive/UvA/M Thesis/Data/Results/Predictions'  # Predictions for test set

In [ ]:
# Mount Drive.
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Copy results to runtime.
parameters_dir = '/content/Parameters'
predictions_dir = '/content/Predictions'

import shutil

shutil.copytree(parameters_drive_dir, parameters_dir)
shutil.copytree(predictions_drive_dir, predictions_dir)

'/content/Predictions'

In [ ]:
# Functions to process results

import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.utils import resample

def bootstrap_metrics(data, threshold, n_bootstrap=100, sample_size=50):
    """
    Calculation of performance metrics based on bootstrapped test samples.
    :param: data: a dataframe containing predictions.
                  Must contain columns 'prediction' and 'label'.
    :param: threshold: the threshold to use for positive predictions.
    :param: n_bootstrap: the number of bootstrapping rounds.
    :param: sample_size: the size of one bootstrapped sample.
    :return: a dictionary containing the accuracy, recall, precision and their standard deviations.
    """
    accuracy_scores = []
    recall_scores = []
    precision_scores = []

    for _ in range(n_bootstrap):
        # Resample the data with replacement
        sample = resample(data, n_samples=sample_size)

        # Convert probabilities to binary predictions based on threshold
        predictions = (sample['prediction'] >= threshold).astype(int)
        true_values = sample['label']

        # Calculate metrics
        accuracy_scores.append(accuracy_score(true_values, predictions))
        recall_scores.append(recall_score(true_values, predictions))
        precision_scores.append(precision_score(true_values, predictions, zero_division=0))

    return {
        'accuracy': (np.mean(accuracy_scores), np.std(accuracy_scores)),
        'recall': (np.mean(recall_scores), np.std(recall_scores)),
        'precision': (np.mean(precision_scores), np.std(precision_scores))
    }

In [ ]:
# Dictionaries for later ease of use.
dataset_dict = {'fishClips' : 'Fish clips', 'AK-fish' : 'AK fish'}
frameSelection_dict = {'evenly' : 'Evenly spaced', 'motionAbsdiff' : 'Motion based'}
dino_dict = {'dinov2-vits14' : 'ViT-S/14', 'dinov2-vitg14' : 'ViT-G/14'}

## AR results

Comparing the performance of our model on the AR task with the baseline model MSQNet

In [ ]:
import os

results = []

for pred_file in os.listdir(predictions_dir):

  # Consider only prediction outputs files.
  if pred_file.startswith('pred'):
    predictions = pd.read_csv(f'{predictions_dir}/{pred_file}')

  else:
    continue

  # Read the setup of the experiment.
  properties = pred_file.split('_')

  dataset = dataset_dict[properties[1]]
  frameSelection = frameSelection_dict[properties[2]]

  model_name = '-'.join(properties[4].split('-')[:2])
  dino_model = dino_dict[model_name]

  # Read threshold for positive predictions.
  params_file = pred_file.replace('pred_', 'params_')
  parameters = pd.read_csv(f'{parameters_dir}/{params_file}')

  pos_threshold = parameters['pos_threshold'][0]

  # Set size of one bootstrapped sample to the size of the prediction set.
  sample_size = len(predictions)

  # Calculate performance metrics based on bootstrapped sample.
  metrics = bootstrap_metrics(predictions, pos_threshold, n_bootstrap, sample_size)

  results.append({
    'Dataset': dataset,
    'Frame selection' : frameSelection,
    'DINOv2 model' : dino_model,
    'Accuracy': f'{metrics["accuracy"][0] * 100:.2f} +/ {1.96 * metrics["accuracy"][1] * 100:.2f} %',
    'Recall': f'{metrics["recall"][0] * 100:.2f} +/ {1.96 * metrics["recall"][1] * 100:.2f} %',
    'Precision': f'{metrics["precision"][0] * 100:.2f} +/ {1.96 * metrics["precision"][1] * 100:.2f} %',
    })

results_df = pd.DataFrame(results)

In [ ]:
# Show the results table.
results_df

,Dataset,Frame selection,DINOv2 model:,Accuracy,Recall,Precision
0,Fish clips,Motion based,ViT-G/14,78.91 +/ 12.05 %,0.00 +/ 0.00 %,0.00 +/ 0.00 %
1,AK fish,Motion based,ViT-S/14,82.65 +/ 4.96 %,34.07 +/ 20.12 %,21.41 +/ 14.84 %
2,Fish clips,Motion based,ViT-S/14,79.73 +/ 11.79 %,0.00 +/ 0.00 %,0.00 +/ 0.00 %


In [ ]:
# Save results table to Drive.
drive_dir = '/content/drive/MyDrive/UvA/M Thesis/Data/Results/Metrics'

if not os.path.exists(drive_dir):
  os.makedirs(drive_dir)

from datetime import datetime
current_date = datetime.now().strftime('%d-%m-%Y')

results_df.to_csv(f'{drive_dir}/AR_results_{current_date}.csv', index=False)